# Stima del costo di un auto
Il costo di mantenimento ordinario di un auto dipende da molte cose (bollo, assicurazione, carburante), di seguito alcuni calcoli provano a stimare le spese per l'acquisto e il mantenimento di un'auto.


In [20]:
import matplotlib.pyplot as plt
from datetime import datetime
import copy

## Passaggio di proprietà
Il passaggio di proprietà, fatto solo in caso di auto usata, dipende dai kW e da una sovratassa locale che varia da provincia

In [ ]:
def passProp(auto: dict, sovra: float) -> float:
    if auto["usata"]:
        return 400
    return 0

## Bollo auto
Il bollo è una spesa annuale che dipende da potenza e classe di efficienza dell'auto.<br>
La tabella per calcolare a mano l'importo del bollo è la seguente:<br>
* Euro 0: € 3 a kW fino a 100 kW e € 4,5 per ogni kW oltre i 100 kW.
* Euro 1: € 2,9 a kW fino a 100 kW e € 4,35 per ogni kW oltre i 100 kW.
* Euro 2: € 2,8 a kW fino a 100 kW e € 4,2 per ogni kW oltre i 100 kW.
* Euro 3: € 2,7 a kW fino a 100 kW e € 4,05 per ogni kW oltre i 100 kW.
* Euro 4, Euro 5 ed Euro 6: € 2,58 a kW fino a 100 kW e € 3,87 per ogni kW oltre i 100 kW.

C'è inoltre da considerare il Superbollo per auto particolarmente potenti.

[Fonte](https://www.alvolante.it/da_sapere/legge-e-burocrazia/bollo-auto-calcolo-scadenza-ed-esenzione-guida-378630)

In [26]:
def calcoloBollo(auto: dict, anno: int) -> float:
    """
    Questa funzione calcola l'importo del bollo di un'auto.

    Parametri:
    auto (dict): L'auto di cui calcolare il bollo.
    anno (int): anno in cui si vuole calcolare il bollo.

    Restituisce:
    Il valore del bollo in euro
    """
    kw = auto["kw"]
    euro = auto["euro"]
    imm = auto["anno"]
    if anno is None or anno<1900:
        anno=imm
    bollo = 0
    c1, c2 = 0, 0
    #definizione dei costi per kW in base alla classe
    if euro == 0:
        c1, c2 = 3, 4,5
    elif euro == 1:
        c1, c2 = 2.9, 4.35
    elif euro == 2:
        c1, c2 = 2.8, 4.2
    elif euro == 3:
        c1, c2 = 2.7, 4.05
    elif euro>3 and euro<7:
        c1, c2 = 2.58, 3.87
    else:
        print("Classe non valida.")
        return 0
    #clacolo dell'importo effettivo
    if kw<=100:
        return c1*kw
    else:
        bollo = c1*100+c2*(kw-100)
        if kw>185:
            superbollo = 20*(kw-185)
            if anno-imm<5:
                bollo = bollo + superbollo
            elif anno-imm<10:
                bollo = bollo + 0.6*superbollo
            elif anno-imm<15:
                bollo = bollo + 0.3*superbollo
            elif anno-imm<20:
                bollo = bollo + 0.15*superbollo
        return bollo

## Costo carburante
Questa variabile non è facilmente trattabile, in quanto il consumo di carburante varia non solo da veicolo a veicolo ma anche da guidatore a guidatore. Inoltre i vari tipi di carburante (GPL, benzina, diesel, metano) hanno concentrazioni di energia diverse e i prezzi cambiano in base al periodo e alla zona.<br>
Di seguito utilizzerò i prezzi medi della mia regione riportati dal [sito ufficiale](https://www.mimit.gov.it/it/prezzo-medio-carburanti/regioni) del ministero competente (modificateli in base alla vostra regione di residenza), mentre per i consumi farò una media dei consumi di vari modelli riportati sul sito [motor1](https://it.motor1.com/reviews/213238/le-auto-che-consumano-meno-la-classifica-del-roma-forli/).

In [22]:
# Dati dei consumi in L/100 km
consumi = [
    3.65, 3.70, 3.70, 3.75, 3.90, 3.90, 3.95, 4.00, 4.00, 4.00, 4.00, 4.05, 4.05, 4.10, 4.10, 4.10,
    4.15, 4.15, 4.15, 4.20, 4.20, 4.25, 4.25, 4.25, 4.25, 4.30, 4.30, 4.30, 4.35, 4.35, 4.35, 4.35,
    4.40, 4.40, 4.45, 4.45, 4.50, 4.50, 4.50, 4.50, 4.55, 4.55, 4.55, 4.60, 4.60, 4.65, 4.70, 4.70,
    4.70, 4.75, 4.75, 4.75, 4.75, 4.80, 4.83, 4.85, 4.90, 4.90, 4.90, 4.95, 5.00, 5.00, 5.00, 5.00,
    5.00, 5.00, 5.00, 5.00, 5.03, 5.05, 5.05, 5.10, 5.10, 5.20, 5.20, 5.20, 5.25, 5.25, 5.25, 5.25,
    5.25, 5.35, 5.35, 5.40, 5.45, 5.48, 5.50, 5.50, 5.55, 5.60, 5.60, 5.80, 6.00, 6.00, 6.00, 6.20,
    6.30, 6.40, 6.40, 6.50, 6.60, 6.70, 6.70, 7.15, 7.35, 7.50, 7.60, 8.45, 10.50
]
# Calcolo della media
media_consumi = sum(consumi) / len(consumi)
cons = {
    "benz":media_consumi
}
print(f"La media dei consumi di un motore a benzina è: {media_consumi:.2f} L/100 km")

consumi = [
    3.00, 3.10, 3.15, 3.28, 3.30, 3.34, 3.35, 3.35, 3.36, 3.40,
    3.40, 3.45, 3.47, 3.50, 3.50, 3.50, 3.60, 3.60, 3.60, 3.60,
    3.63, 3.65, 3.70, 3.75, 3.75, 3.75, 3.80, 3.80, 3.80, 3.85,
    3.85, 3.85, 3.85, 3.90, 3.95, 4.00, 4.00, 4.00, 4.05, 4.05,
    4.10, 4.10, 4.10, 4.10, 4.13, 4.15, 4.20, 4.20, 4.20, 4.20,
    4.20, 4.25, 4.25, 4.30, 4.30, 4.30, 4.30, 4.33, 4.35, 4.35,
    4.37, 4.40, 4.40, 4.40, 4.44, 4.45, 4.45, 4.45, 4.50, 4.50,
    4.50, 4.50, 4.55, 4.55, 4.60, 4.60, 4.60, 4.60, 4.60, 4.65,
    4.65, 4.65, 4.70, 4.70, 4.75, 4.80, 4.80, 4.80, 4.80, 4.80,
    4.80, 4.85, 4.85, 4.90, 4.93, 4.97, 5.00, 5.00, 5.10, 5.10,
    5.10, 5.20, 5.20, 5.20, 5.25, 5.30, 5.30, 5.40, 5.45, 5.45,
    5.45, 5.50, 5.55, 5.70, 5.70, 5.70, 5.73, 5.80, 5.80, 5.80,
    5.90, 5.90, 5.90, 6.00, 6.15, 6.20, 6.40, 7.10, 7.20, 7.20,
    7.30, 7.30, 7.50, 7.50, 7.65, 8.50, 8.90
]
# Calcolo della media
media_consumi = sum(consumi) / len(consumi)
cons["diesel"]=media_consumi
print(f"La media dei consumi di un motore a diesel è: {media_consumi:.2f} L/100 km")

consumi = [
    5.20, 5.50, 5.55, 5.65, 5.85, 6.19, 6.39, 6.45, 6.45, 6.50,
    6.55, 6.60, 6.65, 6.80, 6.80, 6.80, 7.00, 7.18, 7.50, 7.60,
    7.65, 7.70, 7.90, 7.90, 8.20, 8.60
]
# Calcolo della media
media_consumi = sum(consumi) / len(consumi)
cons["gpl"]=media_consumi
print(f"La media dei consumi di un motore a GPL è: {media_consumi:.2f} L/100 km")

consumi = [
    2.55, 2.65, 2.68, 2.90, 2.90, 3.14, 3.15, 3.20, 3.25, 3.30,
    3.40, 3.40, 3.45, 3.60, 3.60, 3.60, 4.06
]
# Calcolo della media
media_consumi = sum(consumi) / len(consumi)
cons["metano"]=media_consumi
print(f"La media dei consumi di un motore a metano è: {media_consumi:.2f} L/100 km")

consumi = [
    3.20, 3.46, 3.55, 3.60, 3.70, 3.70, 3.88, 3.90, 3.95, 3.95,
    4.00, 4.00, 4.00, 4.10, 4.10, 4.10, 4.15, 4.20, 4.20, 4.25,
    4.25, 4.30, 4.35, 4.40, 4.40, 4.40, 4.40, 4.43, 4.45, 4.45,
    4.45, 4.50, 4.50, 4.55, 4.60, 4.65, 4.74, 4.75, 4.80, 4.80,
    4.80, 4.83, 5.00, 5.00, 5.20, 5.20, 5.30, 5.40, 5.40, 5.45,
    5.50, 5.60, 5.85, 6.05, 6.10, 6.10, 6.50, 6.70, 6.85, 7.45

]
# Calcolo della media
media_consumi = sum(consumi) / len(consumi)
cons["ibrida"]=media_consumi
print(f"La media dei consumi di un'auto full hybrid è: {media_consumi:.2f} L/100 km")

La media dei consumi di un motore a benzina è: 5.02 L/100 km
La media dei consumi di un motore a diesel è: 4.70 L/100 km
La media dei consumi di un motore a GPL è: 6.81 L/100 km
La media dei consumi di un motore a metano è: 3.23 L/100 km
La media dei consumi di un'auto full hybrid è: 4.71 L/100 km


In [17]:
def calcoloCarb(auto: dict, km: float=10000) -> float:
    """
    Questa funzione stima la spesa annuale in carburante.

    Parametri:
    auto (dict): L'auto di cui calcolare la spesa per il carburante.
    km (float): kilometri annui che l'auto deve compiere.

    Restituisce:
    La spesa in carburante in euro
    """
    try:
        return km/100*cons[auto["carb"].lower()]
    except:
        print("Carburante non valido.\nLe possibili scelte sono: benzina, diesel, gpl, metano, ibrida")

## Revisione
La revisione (non il tagliando) viene effettuata ogni 2 anni e in caso di auto a GPL o metano vi è da considerare anche il cambio delle bombole

In [8]:
def costoRev(auto: dict, anno: int) -> int:
    """
    Questa funzione calcola il costo della revisione dell'auto.

    Parametri:
    auto (dict): L'auto di cui calcolare la spesa per la revisione.
    anno (int): anno attuale in cui calcolare la revisione.

    Restituisce:
    La spesa in euro per la revisione
    """
    spesa = 0
    if (anno-auto["anno"])%2==0:
        spesa += 80
    
    return spesa

In [16]:
def costoTempo(auto: dict, anni: int=20, sovra: float=0.3, km: float=10000) -> list:
    """
    Questa funzione salva in un array il progredire delle spese per l'auto.

    Parametri:
    auto (dict): L'auto di cui calcolare la spesa nel tempo.
    anni (int): anni per cui si vogliono stimare le spese.
    sovra (float): sovratassa per il passaggio di proprietà

    Restituisce:
    Una lista con il progredire delle spese
    """
    now = datetime.datetime.now().year
    costo = []
    annuale = 0
    for i in range(anni):
        if i==0:
            annuale = auto["costo"]+passProp(auto, sovra)
            costo.append(annuale)
        else:
            annuale = costoRev(auto, now+i)+calcoloCarb(auto, km)
    return costo

In [12]:
bravo = {
    "nome":"bravo",
    "anno":2009,
    "kw":67,
    "costo":4000,
    "usata":True,
    "carb":"GPL",
    "bombole":2019
}

punto = {
    "nome":"punto",
    "anno":2009,
    "kw":57,
    "costo":4000,
    "usata":True,
    "carb":"GPL",
    "bombole":2019
}

costo_bravo = []
costo_punto = []

